# FaIR with input data
## Learning out how to run FaIR with input data. Creating a dummy input dataset to learn formatting
## Created 01/22/2024




In [11]:
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import pooch

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

### Create instance

In [3]:
f = FAIR()

### Define time horizon

In [4]:
# reconstruct hist
f.define_time(1750, 2020, 1)

### Define scenarios

In [6]:
f.define_scenarios(['test1', 'test2'])

### Define configs

In [7]:
f.define_configs(['config1', 'config2'])

### Define species

In [9]:
# going to try just CO2 to start
species = ['CO2']

In [18]:
properties = {
    'CO2': {
        'type': 'co2',
        'input_mode': 'emissions',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False,
    }}

In [19]:
properties

{'CO2': {'type': 'co2',
  'input_mode': 'emissions',
  'greenhouse_gas': True,
  'aerosol_chemistry_from_emissions': False,
  'aerosol_chemistry_from_concentration': False}}

In [20]:
f.define_species(species, properties)

### Create input and output data